In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import os

pio.renderers.default = "notebook_connected"
# pio.renderers.default = "iframe"

# Do not truncate tables
pd.set_option('display.max_columns', None)

In [2]:
PATIENTS_CSV = '../data/patients.csv'
SERVICES_CSV = '../data/services_weekly.csv'
STAFF_CSV = '../data/staff.csv'
SCHEDULE_CSV = '../data/staff_schedule.csv'

df_patients = pd.read_csv(PATIENTS_CSV, delimiter=',', low_memory=False)
df_staff = pd.read_csv(STAFF_CSV, delimiter=',', low_memory=False)
df_staff_schedule = pd.read_csv(SCHEDULE_CSV, delimiter=',', low_memory=False)
df_services_weekly = pd.read_csv(SERVICES_CSV, delimiter=',', low_memory=False)

dfs = [df_patients, df_services_weekly, df_staff, df_staff_schedule]

### Diagram 1

In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 1. Prepare Data
df_services_weekly['availability_status'] = df_services_weekly.apply(
    lambda row: 'Shortage' if row['patients_request'] > row['available_beds'] else 'Sufficient', 
    axis=1
)

# Define metrics and labels
metrics = ['patient_satisfaction', 'staff_morale', 'patients_admitted']
metric_labels = ['Satisfaction', 'Staff Morale', 'Volume (Admitted)']

# Aggregate
df_agg = df_services_weekly.groupby(['service', 'availability_status'])[metrics].mean().reset_index()

# 2. Normalize & Invert
scaler = MinMaxScaler()
df_agg[metrics] = scaler.fit_transform(df_agg[metrics])

# 3. Build Faceted Radar Chart with FIXED SPACING
services_list = df_services_weekly['service'].unique()

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'polar'}]*2]*2, 
    subplot_titles=services_list,
    vertical_spacing=0.30,
    horizontal_spacing=0.15
)

colors = {'Sufficient': '#2ca02c', 'Shortage': '#d62728'}

for i, service in enumerate(services_list):
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    subset = df_agg[df_agg['service'] == service].sort_values('availability_status', ascending=False)
    
    for _, row_data in subset.iterrows():
        status = row_data['availability_status']
        values = row_data[metrics].tolist()
        values += [values[0]] # Close loop
        theta = metric_labels + [metric_labels[0]]
        
        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=theta,
            fill='toself',
            name=status,
            line_color=colors.get(status, '#333'),
            opacity=0.5 if status == 'Sufficient' else 0.7, 
            showlegend=(i==0)
        ), row=row, col=col)

# 4. Final Polish (Fixing Overlaps)
fig.update_layout(
    title_text="<b>Impact of Bed Shortages on Hospital Performance</b><br><i>(Larger Area = Better Performance)</i>",
    height=900, # INCREASED HEIGHT: Makes the whole plot taller
    width=1000,
    legend=dict(title="Bed Status", x=1.05, y=1),
    # Adjust margins to ensure Main Title doesn't hit Subplot Titles
    margin=dict(t=140, b=50, l=80, r=80),
    font=dict(size=11) # Slightly smaller font to help fit labels
)

# Apply explicit range to all subplots
fig.update_polars(radialaxis=dict(visible=True, range=[0, 1]))

# Move Subplot Titles Up slightly (optional fine-tuning)
fig.update_annotations(yshift=30) 

fig.show()

In [15]:
# 1. Load Data
df = pd.read_csv(SERVICES_CSV, delimiter=',', low_memory=False)

# 2. Robust Status Creation (Using simple keys)
# We define the status simply as 'Shortage' or 'Sufficient' to avoid string mismatch bugs
df['availability_status'] = df.apply(
    lambda row: 'Shortage' if row['patients_request'] > row['available_beds'] else 'Sufficient', 
    axis=1
)

# 3. Strict Color Mapping (Fixing the "4 Distinct Colors" issue)
# We assign a fixed integer ID to each service so colors never shift between plots
services = sorted(df['service'].unique()) # ['ICU', 'emergency', 'general_medicine', 'surgery']
service_map = {name: i for i, name in enumerate(services)}
df['service_id'] = df['service'].map(service_map)

# Define 4 Distinct Colors (1 per service)
# 0: Blue, 1: Red, 2: Green, 3: Purple
distinct_colors = ['#1f77b4', '#d62728', '#2ca02c', '#9467bd'] 

# Create a Discrete Colorscale for Plotly ParCoords
# This maps integer 0 -> Color[0], Integer 1 -> Color[1], etc.
colorscale = []
n_services = len(distinct_colors)
for i in range(n_services):
    # Parcoords requires normalized steps (0.0 to 1.0)
    step_start = i / n_services
    step_end = (i + 1) / n_services
    colorscale.append([step_start, distinct_colors[i]])
    colorscale.append([step_end, distinct_colors[i]])

# 4. Normalize Data (Global Scale)
metrics = ['patient_satisfaction', 'staff_morale', 'patients_admitted']
labels = ['Satisfaction', 'Staff Morale', 'Admitted']

scaler = MinMaxScaler()
df_norm = df.copy()
df_norm[metrics] = scaler.fit_transform(df[metrics])

# 5. Create Parallel Coordinates Figures (Separate, not subplots)
def create_parcoords_fig(status_filter, title):
    subset = df_norm[df_norm['availability_status'] == status_filter]
    
    # Check if subset is empty to avoid "Empty Diagram" bug
    if subset.empty:
        print(f"Warning: No data found for status '{status_filter}'")
        return None

    fig_temp = go.Figure(
        data=[go.Parcoords(
            line=dict(
                color=subset['service_id'],
                colorscale=colorscale,
                cmin=0,
                cmax=n_services, # Fixed range ensures colors strictly match the ID
                showscale=True,
                colorbar=dict(
                    title='Service',
                    tickvals=[i + 0.5 for i in range(n_services)], # Center ticks
                    ticktext=services,
                    len=0.6
                )
            ),
            dimensions=[
                dict(range=[0,1], label=l, values=subset[m]) for m, l in zip(metrics, labels)
            ]
        )]
    )
    
    fig_temp.update_layout(
        title_text=title,
        height=500,
        width=1000,
        margin=dict(l=60, r=60, t=80, b=40)
    )
    
    return fig_temp

# 6. Create and display figures separately
fig_sufficient = create_parcoords_fig('Sufficient', "<b>Low Load (Sufficient)</b>")
if fig_sufficient:
    fig_sufficient.show()

fig_shortage = create_parcoords_fig('Shortage', "<b>High Load (Shortage)</b>")
if fig_shortage:
    fig_shortage.show()

In [16]:
# 1. Load Data
df = pd.read_csv(SERVICES_CSV, delimiter=',', low_memory=False)

# 2. Define Status (Low vs High Demand)
df['availability_status'] = df.apply(
    lambda row: 'Shortage' if row['patients_request'] > row['available_beds'] else 'Sufficient', 
    axis=1
)

# 3. Map Status to Color IDs
# 0: Sufficient (Blue), 1: Shortage (Red)
status_map = {'Sufficient': 0, 'Shortage': 1}
df['status_id'] = df['availability_status'].map(status_map)

# Define Distinct Colors: Blue for Good, Red for Bad
distinct_colors = ['#1f77b4', '#d62728'] # Blue, Red

# Create Discrete Colorscale for 2 states
colorscale = [
    [0.0, distinct_colors[0]],
    [0.5, distinct_colors[0]],
    [0.5, distinct_colors[1]],
    [1.0, distinct_colors[1]]
]

# 4. Normalize Data (Global Scale for valid comparison across services)
metrics = ['patient_satisfaction', 'staff_morale', 'patients_admitted', 'patients_refused']
labels = ['Satisfaction', 'Staff Morale', 'Admitted', 'Refusals']

scaler = MinMaxScaler()
df_norm = df.copy()
df_norm[metrics] = scaler.fit_transform(df[metrics])

# 5. Generate 4 Separate Diagrams (One per Service)
services = sorted(df['service'].unique())

for service in services:
    # Filter data for this service
    subset = df_norm[df_norm['service'] == service]
    
    # Skip if empty
    if subset.empty:
        continue
        
    fig = go.Figure(data=go.Parcoords(
        line=dict(
            color=subset['status_id'],
            colorscale=colorscale,
            cmin=0,
            cmax=1, # Range 0-1 ensures 0 maps to Blue, 1 maps to Red
            showscale=True,
            colorbar=dict(
                title='Load Status',
                tickvals=[0.25, 0.75],
                ticktext=['Low Load (Sufficient)', 'High Load (Shortage)'],
                len=0.5,
                thickness=15
            )
        ),
        dimensions=[
            dict(range=[0,1], label=l, values=subset[m]) for m, l in zip(metrics, labels)
        ]
    ))
    
    fig.update_layout(
        title_text=f"<b>Service Analysis: {service.replace('_', ' ').title()}</b><br><i>(Blue = Sufficient Beds, Red = Bed Shortage)</i>",
        height=400,
        width=1000,
        margin=dict(l=60, r=60, t=80, b=40)
    )
    
    fig.show()

### Diagram 2


### Diagram 3


### Diagram 4


### Diagram 5


Task 5 — Identify Clusters of Services with Similar Operational Behavior (Aggregation + Clustering)
Discover groups of similar weekly patterns to take broader strategic decisions (e.g., which services suffer together during flu season).
 This directly aligns with Assignment 5 – Aggregation/Clustering.
Relevant Attributes (services_weekly):
available_beds


patients_request


patients_admitted


staff_morale


patient_satisfaction


event (convert to numeric encoding)


Aggregation & Clustering Approach:
Weekly vectors for each service (52 rows per service)


Perform k-means clustering to identify patterns (high-demand vs low-satisfaction weeks) 
Group weeks or services into clusters


Suggested Visualization:
Cluster Summary Plots (Aggregation Visualizations):
Use boxplots showing distribution of attributes per cluster


Cluster-level radar chart showing centroid


Color-coded timeline marking cluster membership per week


Useful to see: “Cluster 2” weeks were high-demand/low-satisfaction weeks → often correspond with flu or strikes.
